# **LAB THREE: SELECTIVE EXCITATION**

This lab covers...

## Lab Structure

1. Projection Images
Motivation: Why do we want to slice select?

Experiment 1.1: uses three tube phantom and shows them overlapping

2. Frequency Selective Excitation

Theory: How does a gradient during an RF pulse affect which spins are excited? Show equation relating between frequency range, gradient strength and thickness of excited region.

Theory: what is a soft pulse vs hard pulse? Mention inverse relation of pulse width with bandwidth.

Experiment 2.1: Adjust pulse width with a continuous gradient and see how the range of frequencies excited changes.

Question: What is the resulting slice profile from a long rectangular pulse?

Experiment 2.2: 2D RARE slice - uses rectangle pulse with same width as experiment 2.1. Also uses three tube phantom.

3. Slice Profile Theory

Theory: How do we excite all the spins in the slice equally, i.e. get a flat slice profile?
FT of sinc function is a rectangle.
Sinc function is infinite and the pulse needs to be short compared to T1/T2 relaxation, what happens when the RF pulse is truncated?
Apodization: mutliply by gaussian to smooth the ends of the pulse and reduce ripple in the slice profile.

4. Soft Pulse Design

Experiment 4.1: Adjust width/lobes/apodisation and see how the frequency spectrum changes.

Show pulse sequence for experiment 4.2, explain the need for the rephasing gradient pulse after the slice select pulse.
Experiment 4.2: Run the slice profile measurement MRI sequence.
Question: How does the slice profile compare to the calculated frequency spectrum?

Experiment 4.3: 2D RARE similar to 2.2. 

5. SLR Soft Pulse Design

Theory: Ripples on the sides and top of slice profile and how they can be optimised. Why they need to be. Allows you to directly specify the bandwidth. Used in ilumr

Experiment 6.1: Set SLR pulse width, bandwidth, and flip angle. See the pulse shape, spectrum, and results of slice profile measurement.

6. Controlling slice position
Equation, diagram, brief explanation
Adjust gradient to achieve slice width using RF pulse designed above.
Pulse Sequence: 2D image with slice selection using soft 90.
Experiment: 2D slice using student designed pulse with frequency offset input - three tube phantom.
Experiment: Finding the ilumr text 

>-------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Setup Task: Run the Notebook**
> 
> 1. Edit the cell below to set the `LAB_USER_NAME` variable to your name
> 2. Click **Run->Run All Cells** in the in top menu bar of jupyterlab
> 3. Open the Table of Contents side-bar on the left edge of jupyterlab to aid in navigation
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
LAB_USER_NAME = 'Sharon'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar.

In [2]:
import panel as pn
pn.extension(raw_css=['''progress {margin: 0;}''']) # raw_css setting is a workaround for panel issue 4112
import sys
import os
import numpy as np

# add inline dashboard libraries to path so they can be imported later
sys.path.append('../../../dashboards-inline')

LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('User data directory:', LAB_DIR)

User data directory: /home/data/Sharon


## 1. Projection Images
Thus far, the 2D image created in the labs have been projection images. These images are created using a rectangular RF-pulse (known as a hard pulse) and both frequency and phase encoding gradients (Fig.1,a). The frequency and phase encoding gradients allow the aquired signals to be sorted into 2D space to produce an image. The orientation of the image will depend on which axes that the frequency and phase gradients were applied along. Since there is no form of encoding in the third dimension of the sample, the signals along this axis will be summed in the 2D image.   

<center><img src="Images/projection2.0.png" width="1300"></center>
<center><figcaption style="width: 600px;">Figure 1: Setting the frequency and phase encoding in the x and z axes produces a projection image where the varying signal components in the y direction are summed into average values.  </figcaption></center>

TODO: swap z and y axes
TODO: add acquisition gating waveform

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 1.1: Generate a Projection Image**
> 1. Insert the mystery sample (ilumr sample) at the correct depth.
> 2. Choose a the phase encoding and frequency encoding axes and start the experiment by pressing "Run". 
>
> **Question:** Can you read the text located in the centre of the phantom?
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [3]:
# Experiment 1.1: 2D RARE projection image experiment with selectable phase and frequency encoding axis

RESOLUTION = 64

AXIS_VECTORS = {
    'X': np.array([1,0,0], dtype=float),
    'Y': np.array([0,1,0], dtype=float),
    'Z': np.array([0,0,1], dtype=float)
}

f_axis_input = pn.widgets.RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='Z', width = 200)
p_axis_input = pn.widgets.RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width = 200)

def get_g_phase():
    g_phase_max = 0.25
    g_phase_prop = np.linspace(1, -1, RESOLUTION, endpoint=False)
    return np.outer(g_phase_max*g_phase_prop, AXIS_VECTORS[p_axis_input.value])

def get_g_read():
    g_read_max = 0.25
    return g_read_max*AXIS_VECTORS[f_axis_input.value]

import importlib
import RARE2D
importlib.reload(RARE2D)

from RARE2D import RARE2DApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=get_g_read,
    g_phase_1=get_g_phase,
    n_ETL=RESOLUTION,
    t_echo=0.01,
    n_scans=1,
    n_samples=RESOLUTION,
    t_dw=20e-6,
    t_end=1,
)

override_pars['t_read'] = override_pars['t_dw']*override_pars['n_samples']
override_pars['t_phase'] = override_pars['t_read']/2

# create dashboard app
exp1_app = RARE2DApp(
    override_pars=override_pars
)

# display layout
pn.Column(
    pn.Row(pn.pane.HTML('<label>Frequency Encoding Axis</label>', width=200),f_axis_input),
    pn.Row(pn.pane.HTML('<label>Phase Encoding Axis</label>', width=200),p_axis_input), 
    exp1_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Row
        [0] HTML(str, width=200)
        [1] RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='Z', width=200)
    [1] Row
        [0] HTML(str, width=200)
        [1] RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width=200)
    [2] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [2] Progress(height=31, sizing_mode='stretch_width', value=0)
            [3] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

## 2. Frequency Selective Excitation

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.1: Pulse Width**
> 1. TODO **Question: What is the resulting slice profile from a long rectangular pulse?**
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:

from SE_const_grad_app import ConstGradSEApp # from dashboards-inline directory that was added to sys.path

AREA_90 = 0.3*32e-6

exp2_width_input = pn.widgets.FloatInput(name="width (μs)", start=50, end=2000, step=50, value=50, width=80)

# set some parameters directly
override_pars = dict(
    a_90=lambda: AREA_90/(exp2_width_input.value*1e-6),
    t_90=lambda: exp2_width_input.value*1e-6,
    g_read=(0,0,0.25),
    n_scans=1,
    n_samples=200,
    t_dw=40e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5,
    t_echo = 10e-3
)

# create dashboard app
exp2_app = ConstGradSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=False,
    enable_run_loop=True,
    flat_filter = True
)

exp2_app.plot1.figure.height=400
exp2_app.plot2.figure.height=400

# display layout
pn.Column(
    exp2_width_input,
    exp2_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] FloatInput(end=2000, name='width (μs)', start=50, step=50, value=50, width=80)
    [1] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.2: 2D RARE using rectangular slice pulse**
> 1. TODO: show why bad
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Experiment 1.1: 2D RARE projection image experiment with selectable phase and frequency encoding axis

RESOLUTION = 64

AXIS_VECTORS = {
    'X': np.array([1,0,0], dtype=float),
    'Y': np.array([0,1,0], dtype=float),
    'Z': np.array([0,0,1], dtype=float)
}

f_axis_input = pn.widgets.RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='Z', width = 200)
p_axis_input = pn.widgets.RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width = 200)

def get_g_phase():
    g_phase_max = 0.25
    g_phase_prop = np.linspace(1, -1, RESOLUTION, endpoint=False)
    return np.outer(g_phase_max*g_phase_prop, AXIS_VECTORS[p_axis_input.value])

def get_g_read():
    g_read_max = 0.25
    return g_read_max*AXIS_VECTORS[f_axis_input.value]

import importlib
import RARE2D
importlib.reload(RARE2D)

from RARE2D import RARE2DApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=get_g_read,
    g_phase_1=get_g_phase,
    n_ETL=RESOLUTION,
    t_echo=0.01,
    n_scans=1,
    n_samples=RESOLUTION,
    t_dw=20e-6,
    t_end=1,
)

override_pars['t_read'] = override_pars['t_dw']*override_pars['n_samples']
override_pars['t_phase'] = override_pars['t_read']/2

# create dashboard app
exp1_app = RARE2DApp(
    override_pars=override_pars
)

# display layout
pn.Column(
    pn.Row(pn.pane.HTML('<label>Frequency Encoding Axis</label>', width=200),f_axis_input),
    pn.Row(pn.pane.HTML('<label>Phase Encoding Axis</label>', width=200),p_axis_input), 
    exp1_app.main(),
    sizing_mode='stretch_width'
)

## 3. Slice Profile Theory 

## 4. Soft Pulse Design

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 4.1: Visualising Sinc Pulse Design**
>The experiment below is designed to help you visualise how changing the parameters of a soft pulse changes the profile in the frequency domain.(explain what things cause issues e.g. ripples on the top of the spectrum and what this does to the final image).The Rf-Pulse you design will be used for the following experiments.
> 1. Experiment with the three soft pulse parameters and observe how the shape of the RF-pulse and slice profile changes. 
> 2. **Question: With the other inputs left unchanged, how does changing the pulse width affect the spectral bandwidth?**
> 3. **Question: How does adding more lobes improve the shape of the slice profile? How must the pulse width be adjusted to keep the spectral bandwidth constant?**
> 4. Apodisation can be used to reduce the ripple in the slice profile. **Question: What is the trade-off?** Pick a value that looks like a good compromise.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
from matipo.util.pulseshape import calc_soft_pulse
from matipo.util.plots import SharedXPlot, ComplexPlot
from scipy.fft import fft,fftfreq, fftshift, ifftshift
from bokeh.plotting import figure

exp4_width_input = pn.widgets.FloatInput(name="duration (μs)", start=1, end=2000, step=10, value=400, width=80)
exp4_n_lobe_input = pn.widgets.FloatInput(name="lobes", start=0, end=100, step=1, value=3, width=80)
exp4_apodization_input = pn.widgets.FloatInput(name="apodization", start=0, end=100, step=1, value=0, width=80)

exp4_p1 = ComplexPlot(
    title="Waveform",
    x_axis_label="Time (s)",
    y_axis_label="Amplitude",
    height=400)

exp4_p2 = ComplexPlot(
    title="Spectrum",
    x_axis_label="Frequency (Hz)",
    y_axis_label="Spectral Density",
    height=400)

plot_row = pn.Row(exp4_p1.figure, exp4_p2.figure, sizing_mode='stretch_width')

global_shape = [1]
global_width = 100e-6

def update_plots(event):
    global global_width, global_shape 
    width = exp4_width_input.value*1e-6
    N = int(width / 1e-6)
    if N > 1000:
        N = 1000
    dt = width / N
    pts = np.sinc(np.linspace(-exp4_n_lobe_input.value, exp4_n_lobe_input.value,N)) 
    t = np.arange(N)*dt
    t_0 = (width-dt)/2
    pts*=np.exp(-exp4_apodization_input.value*(np.linspace(-1, 1, N)**2))
    global_shape = pts
    global_width = width
    freq = fftfreq(N,dt)
    spectrum = fft(pts)
    spectrum *= np.exp(-1j*np.pi*(width+dt)*freq) # fix phase of spectrum plot due to time offset
    freq = fftshift(freq)
    spectrum = fftshift(spectrum)
    exp4_p1.update_data(t, pts)
    exp4_p2.update_data(freq, spectrum)
    pn.io.push_notebook(plot_row)

# update plot when any value is changed
exp4_width_input.param.watch(update_plots, 'value')
exp4_n_lobe_input.param.watch(update_plots, 'value')
exp4_apodization_input.param.watch(update_plots, 'value')

# manually trigger for first update
exp4_width_input.param.trigger('value')

app = pn.Row(
    pn.Column(exp4_width_input, exp4_n_lobe_input, exp4_apodization_input),
    plot_row,
    sizing_mode='stretch_width')

app

Row(sizing_mode='stretch_width')
    [0] Column
        [0] FloatInput(end=2000, name='duration (μs)', start=1, step=10, value=400, width=80)
        [1] FloatInput(end=100, name='lobes', start=0, step=1, value=3, width=80)
        [2] FloatInput(end=100, name='apodization', start=0, step=1, width=80)
    [1] Row(sizing_mode='stretch_width')
        [0] Bokeh(Figure)
        [1] Bokeh(Figure)

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 4.2: 1D Slice Profile**
> With the sinc pulse designed, we can now use it to create a 1D image of the slice profile. Later in the lab we will explore the effect the slice-select gradient has on the slice profile. For now, the gradient strength ($G_{ss}$) is fixed at 50% (add the actual value of this for calculations).    
> 1. Based on this value and the frequency range ($\Delta$F) of the sinc pulse you have designed, calculate the expected thickness of the slice profile.
> 2. Insert the shim sample (make sure to centre the sample using the depth gauge).
> 3. Run the experiment and check your answer against the results.
> 4. Change the shape of the RF-pulse and observe how this changes the slice profile. **Question:** How does the slice profile compare to the calculated frequency spectrum?
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
from custom_pulse_SE_app import CustomPulseSEApp # from dashboards-inline directory that was added to sys.path

def estimate_a_90():
    shape_area = np.mean(global_shape)*global_width
    target_area = 0.3*32e-6 # TODO: load from hardpulse_90.yaml
    return min(1, abs(target_area/shape_area))

# set some parameters directly
override_pars = dict(
    a_90=estimate_a_90,
    t_90=lambda: global_width,
    shape_90=lambda: global_shape,
    g_slice=(0,0,-0.5),
    g_read=(0,0,0.5),
    n_scans=2,
    n_samples=200,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5
)

# create dashboard app
exp4_2_app = CustomPulseSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True,
    flat_filter = True
)

exp4_2_app.plot1.figure.height=400
exp4_2_app.plot2.figure.height=400

# display layout
pn.Column(
    exp4_2_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 4.3: 2D RARE**
> 1. TODO
> -------------------------------------------------------------------------------------------------------------------------------------------------------

## 5. SLR Soft Pulse Design

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 5.1: SLR**
> 1. TODO 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
from matipo.util.pulseshape import calc_soft_pulse
from matipo.util.plots import SharedXPlot, ComplexPlot
from scipy.fft import fft,fftfreq, fftshift, ifftshift
from bokeh.plotting import figure

exp6_width_input = pn.widgets.FloatInput(name="duration (s)", start=0, end=600e-6, step=1e-6, value=375e-6, width=80)
exp6_bandwidth_input = pn.widgets.FloatInput(name="bandwidth", start=0, end=20000, step=10, value=13333, width=80)
exp6_amp_input = pn.widgets.FloatInput(name="amplitude", start=0, end=1, step=0.001, value=0.2818, width=80)

p1 = ComplexPlot(
    title="Waveform",
    x_axis_label="Time (s)",
    y_axis_label="Amplitude",
    height=400)

p2 = ComplexPlot(
    title="Spectrum",
    x_axis_label="Frequency (Hz)",
    y_axis_label="Spectral Density",
    height=400)

plot_row = pn.Row(p1.figure, p2.figure, sizing_mode='stretch_width')

global_slr_shape = [1]
global_slr_width = 100e-6
global_slr_amp = 1

def update_plots(event):
    global global_slr_width, global_slr_shape, global_slr_amp 
    N, dt, pts = calc_soft_pulse(exp6_width_input.value, exp6_bandwidth_input.value)
    global_slr_width = exp6_width_input.value
    global_slr_shape = pts
    global_slr_amp = exp6_amp_input.value
    pts *= exp6_amp_input.value
    t = np.arange(N)*dt*1000
    spectrum = fft(fftshift(pts))
    freq = fftfreq(N,dt)
    freq = fftshift(freq)
    spectrum = fftshift(spectrum)
    p1.update_data(t, pts)
    p2.update_data(freq, spectrum)
    pn.io.push_notebook(plot_row)

# global_shape = [1]
# global_width = 100e-6

# def update_plots(event):
#     global global_width, global_shape 
#     width = width_input.value*1e-6
#     N = int(width / 1e-6)
#     if N > 1000:
#         N = 1000
#     dt = width / N
#     pts = np.sinc(np.linspace(-n_lobe_input.value, n_lobe_input.value,N)) 
#     t = np.arange(N)*dt
#     t_0 = (width-dt)/2
#     pts*=np.exp(-apodization_input.value*(np.linspace(-1, 1, N)**2))
#     global_shape = pts
#     global_width = width
#     freq = fftfreq(N,dt)
#     spectrum = fft(pts)
#     spectrum *= np.exp(-1j*np.pi*(width+dt)*freq) # fix phase of spectrum plot due to time offset
#     freq = fftshift(freq)
#     spectrum = fftshift(spectrum)
#     p1.update_data(t, pts)
#     p2.update_data(freq, spectrum)
#     pn.io.push_notebook(plot_row)

# update plot when any value is changed
exp6_width_input.param.watch(update_plots, 'value')
exp6_bandwidth_input.param.watch(update_plots, 'value')
exp6_amp_input.param.watch(update_plots, 'value')

# manually trigger for first update
exp6_width_input.param.trigger('value')

app = pn.Row(
    pn.Column(exp6_width_input, exp6_bandwidth_input, exp6_amp_input),
    plot_row,
    sizing_mode='stretch_width')

app

Row(sizing_mode='stretch_width')
    [0] Column
        [0] FloatInput(end=0.0006, name='duration (s)', start=0, step=1e-06, value=0.000375, width=80)
        [1] FloatInput(end=20000, name='bandwidth', start=0, step=10, value=13333, width=80)
        [2] FloatInput(end=1, name='amplitude', start=0, step=0.001, value=0.2818, width=80)
    [1] Row(sizing_mode='stretch_width')
        [0] Bokeh(Figure)
        [1] Bokeh(Figure)

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 5.2: SLR**
> 1. TODO: Get them to adjust values in the previous experiment and see the results 
> 2. Provide values that will give a good slice. Different phantoms to show how signal and spectrum change while the RF pulse is kept constant 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
from custom_pulse_SE_app import CustomPulseSEApp # from dashboards-inline directory that was added to sys.path

def estimate_a_90():
    shape_area = np.mean(global_shape)*global_width
    target_area = 0.3*32e-6 # TODO: load from hardpulse_90.yaml
    return min(1, abs(target_area/shape_area))

# set some parameters directly
override_pars = dict(
    a_90=lambda: global_slr_amp,
    t_90=lambda: global_slr_width,
    shape_90=lambda: global_slr_shape,
    g_slice=(0,0,-0.5),
    g_read=(0,0,0.5),
    n_scans=2,
    n_samples=200,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.5
)

# create dashboard app
exp6_2_app = CustomPulseSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True,
    flat_filter = True
)

exp6_2_app.plot1.figure.height=400
exp6_2_app.plot2.figure.height=400

# display layout
pn.Column(
    exp6_2_app.main(),
    sizing_mode='stretch_width'
)

Column(sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_both')
        [0] Row(sizing_mode='stretch_both')
            [0] Bokeh(Figure)
            [1] Bokeh(Figure)
        [1] Row(sizing_mode='stretch_width')
            [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
            [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
            [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
            [3] Progress(height=31, sizing_mode='stretch_width', value=0)
            [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 5.3: 2D RARE**
> 1. TODO
> -------------------------------------------------------------------------------------------------------------------------------------------------------

## 6. Controlling Slice Position

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 6.1: Locating the Hidden Text**
> TO DO: insert a standard 2D RARE dashboard where they can play around with slice selection in different axes to find the ilumr text in the ilumr phantom
> 1. Change the...
> -------------------------------------------------------------------------------------------------------------------------------------------------------